In [1]:
import os
import certifi
os.environ["REQUESTS_CA_BUNDLE"] = certifi.where()
os.environ["SSL_CERT_FILE"] = certifi.where()

In [6]:
#lang parsing 
import re
from lingua import Language, LanguageDetectorBuilder

#hf lib
import datasets
from datasets import load_dataset

#multiprocessing lib
import multiprocessing
import gc

# Detect Chinese

In [8]:
#detect chinese
def contains_chinese_characters(text):
    pattern = re.compile(r'[\u4e00-\u9fff]+')
    return bool(pattern.search(text))

# Get Dataset info

In [15]:
dataset_info = datasets.load_dataset_builder("tiiuae/falcon-refinedweb")
files= dataset_info.config.data_files["train"][0]
print("First file:", dataset_info.config.data_files["train"][0])
print("Dataset hash:", dataset_info.hash)

Resolving data files:   0%|          | 0/5534 [00:00<?, ?it/s]

First file: hf://datasets/tiiuae/falcon-refinedweb@c735840575b629292b41da8dde11dcd523d4f91c/data/train-00000-of-05534-b8fc5348cbe605a5.parquet
Dataset hash: c735840575b629292b41da8dde11dcd523d4f91c


In [16]:
first_2_shards=dataset_info.config.data_files["train"][:3]
first_2_shards

['hf://datasets/tiiuae/falcon-refinedweb@c735840575b629292b41da8dde11dcd523d4f91c/data/train-00000-of-05534-b8fc5348cbe605a5.parquet',
 'hf://datasets/tiiuae/falcon-refinedweb@c735840575b629292b41da8dde11dcd523d4f91c/data/train-00001-of-05534-9bca3ce859516338.parquet',
 'hf://datasets/tiiuae/falcon-refinedweb@c735840575b629292b41da8dde11dcd523d4f91c/data/train-00002-of-05534-01680948bd81de83.parquet']

In [5]:
#load entire dataset
falcon_dataset = load_dataset("tiiuae/falcon-refinedweb", data_files=files, streaming=True)
falcon_dataset = falcon_dataset["train"]
#falcon_dataset = iter(falcon_dataset["train"])

# with preload dataset

In [22]:
falcon_dataset1 = load_dataset("tiiuae/falcon-refinedweb", data_files='hf://datasets/tiiuae/falcon-refinedweb@c735840575b629292b41da8dde11dcd523d4f91c/data/train-00000-of-05534-b8fc5348cbe605a5.parquet', streaming=True)
falcon_dataset2 = load_dataset("tiiuae/falcon-refinedweb", data_files='hf://datasets/tiiuae/falcon-refinedweb@c735840575b629292b41da8dde11dcd523d4f91c/data/train-00002-of-05534-01680948bd81de83.parquet', streaming=True)
two_shards_list = [falcon_dataset1, falcon_dataset2]

In [23]:
def iterate_corpus_for_multi(shared_list, shard):
    text_list = []
    falcon_dataset_shard = shard["train"]
    for index, row_text in enumerate(falcon_dataset_shard):
        if contains_chinese_characters(row_text["content"]):
            text_list.append(row_text)
    return shared_list.append(text_list)

In [24]:
def multi(file_list, CPU):
    if __name__ == '__main__':
        print('Running... Multi')
        manager_list = multiprocessing.Manager().list()
        processes= []

        for shard in file_list:
            p = multiprocessing.Process(target= iterate_corpus_for_multi, args=(manager_list, shard))
            p.start()
            processes.append(p)
        ended = False

        while not ended:
            ended=True
            for p in processes:
                p.join(15) 
                if (p.is_alive()):
                  ended = False
        return manager_list

In [ ]:
%time res = multi(two_shards_list, CPU=2)

Running... Multi


# Iterate 1 shard template

In [7]:
def iterate_corpus(shard):
    text_list = []
    for index, row_text in enumerate(shard):
        if contains_chinese_characters(row_text["content"]):
            text_list.append(row_text)
    return text_list

In [8]:
%time test_res = iterate_corpus(iterable_shard)

CPU times: user 2min 10s, sys: 31.7 s, total: 2min 41s
Wall time: 8min 58s


In [10]:
test_res[0]

{'content': "Discover Your Blip. What do you want to watch?\nToday's song is called Just a Waste of Time. ;Listen to it on MySpace: YouTube: script:...日本語訳＋番組詳細 1293 Friday 3 SeptemberThe Daily English Show\nThe Daily English Show is the world's first daily online English language show. Produced and presented by Sarah - a New Zealander based in Auckland, New Zealand.",
 'url': 'http://blip.tv/thedailyenglishshow/1293-just-a-waste-of-time-negative-news-scheme-vs-scam-glad-wrap-4102793',
 'timestamp': datetime.datetime(2013, 5, 18, 10, 32, 1),
 'dump': 'CC-MAIN-2013-20',
 'segment': '1368696382261',
 'image_urls': []}

In [17]:
len(test_res)

655

# 2 SHARDS (DONT TOUCH)

In [3]:
def iterate_corpus_for_multi(shared_list, shard_name):
    text_list = []
    falcon_dataset_shard = load_dataset("tiiuae/falcon-refinedweb", data_files=shard_name, streaming=True)["train"]
    for index, row_text in enumerate(falcon_dataset_shard):
        if contains_chinese_characters(row_text["content"]):
            text_list.append(row_text)
    return shared_list.append(text_list)

In [4]:
def multi(file_list, CPU):
    if __name__ == '__main__':
        print('Running... Multi')
        manager_list = multiprocessing.Manager().list()
        processes= []

        for shard_name in file_list:
            p = multiprocessing.Process(target= iterate_corpus_for_multi, args=(manager_list, shard_name))
            p.start()
            processes.append(p)
        ended = False

        while not ended:
            ended=True
            for p in processes:
                p.join(15) 
                if (p.is_alive()):
                  ended = False
        return manager_list

In [9]:
file_list = ['hf://datasets/tiiuae/falcon-refinedweb@c735840575b629292b41da8dde11dcd523d4f91c/data/train-00000-of-05534-b8fc5348cbe605a5.parquet',
 'hf://datasets/tiiuae/falcon-refinedweb@c735840575b629292b41da8dde11dcd523d4f91c/data/train-00001-of-05534-9bca3ce859516338.parquet']
%time res = multi(file_list, CPU=2)

Running... Multi
CPU times: user 46 ms, sys: 37 ms, total: 83 ms
Wall time: 10min 36s


In [13]:
len(res[:][1])

655

# 3 SHARDS

In [17]:
def iterate_corpus_for_multi(shared_list, shard_name):
    text_list = []
    falcon_dataset_shard = load_dataset("tiiuae/falcon-refinedweb", data_files=shard_name, streaming=True)["train"]
    for index, row_text in enumerate(falcon_dataset_shard):
        if contains_chinese_characters(row_text["content"]):
            text_list.append(row_text)
    return shared_list.append(text_list)

In [18]:
def multi(file_list, CPU):
    if __name__ == '__main__':
        print('Running... Multi')
        manager_list = multiprocessing.Manager().list()
        processes= []

        for shard_name in file_list:
            p = multiprocessing.Process(target= iterate_corpus_for_multi, args=(manager_list, shard_name))
            p.start()
            processes.append(p)
        ended = False

        while not ended:
            ended=True
            for p in processes:
                p.join(15) 
                if (p.is_alive()):
                  ended = False
        return manager_list

In [19]:
file_list = ['hf://datasets/tiiuae/falcon-refinedweb@c735840575b629292b41da8dde11dcd523d4f91c/data/train-00000-of-05534-b8fc5348cbe605a5.parquet',
 'hf://datasets/tiiuae/falcon-refinedweb@c735840575b629292b41da8dde11dcd523d4f91c/data/train-00001-of-05534-9bca3ce859516338.parquet',
 'hf://datasets/tiiuae/falcon-refinedweb@c735840575b629292b41da8dde11dcd523d4f91c/data/train-00002-of-05534-01680948bd81de83.parquet']
%time res = multi(file_list, CPU=3)

Running... Multi
CPU times: user 79.7 ms, sys: 100 ms, total: 180 ms
Wall time: 15min 34s


# MULTISHARD

In [50]:
#vers 1
# iterate multiple setup
def iterate_corpus_for_multi(shared_list, shard_name):
    text_list = []
    falcon_dataset_shard = load_dataset("tiiuae/falcon-refinedweb", data_files=shard_name, streaming=True)["train"]
    for index, row_text in enumerate(falcon_dataset_shard):
        if contains_chinese_characters(row_text["content"]):
            text_list.append(row_text)
    return shared_list.append(text_list)

def multi(file_list, CPU=1):
    if __name__ == '__main__':
        print('Running... Multi')
        manager_list = multiprocessing.Manager().list()
        processes= []

        for shard_name in file_list:
            p = multiprocessing.Process(target= iterate_corpus_for_multi, args=(manager_list, shard_name))
            p.start()
            processes.append(p)
        ended = False

        while not ended:
            ended=True
            for p in processes:
                # print(f'{p=}')
                p.join(15)  # check every 15 sec
                if (p.is_alive()):
                  ended = False
                # print(f'{p=} {p.is_alive()=}')
        return manager_list
file_list = ['hf://datasets/tiiuae/falcon-refinedweb@c735840575b629292b41da8dde11dcd523d4f91c/data/train-00000-of-05534-b8fc5348cbe605a5.parquet',
 'hf://datasets/tiiuae/falcon-refinedweb@c735840575b629292b41da8dde11dcd523d4f91c/data/train-00001-of-05534-9bca3ce859516338.parquet']
%time res = multi(file_list, CPU=2)

Running... Multi
CPU times: user 86 ms, sys: 123 ms, total: 209 ms
Wall time: 17min 30s


In [ ]:
#vers2
# iterate multiple setup
from datetime import datetime
#lang parsing 
import re
from lingua import Language, LanguageDetectorBuilder

#hf lib
import datasets
from datasets import load_dataset

#multiprocessing lib
import multiprocessing
import gc

#detect chinese
def contains_chinese_characters(text):
    pattern = re.compile(r'[\u4e00-\u9fff]+')
    return bool(pattern.search(text))

def iterate_corpus_for_multi(shared_list, shard_name):
    text_list = []
    falcon_dataset_shard = load_dataset("tiiuae/falcon-refinedweb", data_files=shard_name, streaming=True)["train"]
    for index, row_text in enumerate(falcon_dataset_shard):
        if contains_chinese_characters(row_text["content"]):
            text_list.append(row_text)
    return shared_list.append(text_list)

def multi(file_list, CPU=1):
    print('Running... Multi')
    start_time = datetime.now()
    manager_list = multiprocessing.Manager().list()
    processes= []

    for shard_name in file_list:
        p = multiprocessing.Process(target= iterate_corpus_for_multi, args=(manager_list, shard_name))
        p.start()
        processes.append(p)
    ended = False

    while not ended:
        ended=True
        for p in processes:
            # print(f'{p=}')
            p.join(15)  # check every 15 sec
            if (p.is_alive()):
              ended = False
            # print(f'{p=} {p.is_alive()=}')
        
    return manager_list
        

        
file_list = ['hf://datasets/tiiuae/falcon-refinedweb@c735840575b629292b41da8dde11dcd523d4f91c/data/train-00000-of-05534-b8fc5348cbe605a5.parquet', 'hf://datasets/tiiuae/falcon-refinedweb@c735840575b629292b41da8dde11dcd523d4f91c/data/train-00001-of-05534-9bca3ce859516338.parquet']

if __name__ == '__main__':
    start_time = datetime.now()
    res = multi(file_list, CPU=2)
    end_time = datetime.now()
    print(f'* Start = {start_time}')
    print(f'* End   = {end_time}')
    time_difference = end_time - start_time
    print(f'\n{time_difference = }')
    print(len(res[:]))


Running... Multi


Process Process-24:
Traceback (most recent call last):
  File "/home/cdongjun/.local/lib/python3.8/site-packages/urllib3/response.py", line 712, in _error_catcher
    yield
  File "/home/cdongjun/.local/lib/python3.8/site-packages/urllib3/response.py", line 812, in _raw_read
    data = self._fp_read(amt) if not fp_closed else b""
  File "/home/cdongjun/.local/lib/python3.8/site-packages/urllib3/response.py", line 797, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "/home/cdongjun/anaconda3/envs/test/lib/python3.8/http/client.py", line 459, in read
    n = self.readinto(b)
  File "/home/cdongjun/anaconda3/envs/test/lib/python3.8/http/client.py", line 503, in readinto
    n = self.fp.readinto(b)
  File "/home/cdongjun/anaconda3/envs/test/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  File "/home/cdongjun/anaconda3/envs/test/lib/python3.8/ssl.py", line 1274, in recv_into
    return self.read(nbytes, buffer)
  

In [49]:
len(res[:][1])

548

In [25]:
from datasets import load_dataset
falcon_dataset = load_dataset("tiiuae/falcon-refinedweb", data_files=first_2_shards, streaming=True)
#falcon_dataset = iter(falcon_dataset["train"])

In [29]:
falcon_dataset["train"]

IterableDataset({
    features: ['content', 'url', 'timestamp', 'dump', 'segment', 'image_urls'],
    n_shards: 2
})

In [35]:
%time a = multi(first_2_shards, 2)

Running... Multi


Process Process-6:
Traceback (most recent call last):
  File "/home/cdongjun/anaconda3/envs/test/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/cdongjun/anaconda3/envs/test/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
TypeError: iterate_corpus() takes 1 positional argument but 2 were given
Process Process-7:
Traceback (most recent call last):
  File "/home/cdongjun/anaconda3/envs/test/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/cdongjun/anaconda3/envs/test/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
TypeError: iterate_corpus() takes 1 positional argument but 2 were given


CPU times: user 24.7 ms, sys: 68.3 ms, total: 93 ms
Wall time: 158 ms


In [36]:
a[:]

[]

In [5]:
dataset_info = datasets.load_dataset_builder("tiiuae/falcon-refinedweb")
files= dataset_info.config.data_files["train"][0]
print("First file:", dataset_info.config.data_files["train"][0])
print("Dataset hash:", dataset_info.hash)

Resolving data files:   0%|          | 0/5534 [00:00<?, ?it/s]

First file: hf://datasets/tiiuae/falcon-refinedweb@c735840575b629292b41da8dde11dcd523d4f91c/data/train-00000-of-05534-b8fc5348cbe605a5.parquet
Dataset hash: c735840575b629292b41da8dde11dcd523d4f91c


In [36]:
files= dataset_info.config.data_files["train"][0]

In [ ]:
a.num_rows

In [104]:
def contains_chinese_characters(text):
    pattern = re.compile(r'[\u4e00-\u9fff]+')
    return bool(pattern.search(text))
print(contains_chinese_characters("abca是"))

True


In [74]:
def iterate_corpus():
    text_list = []
    for index, content in enumerate
            
    for index,text in enumerate(range(2)):
        temp = next(falcon_dataset)
        #content = temp["content"] + temp["url"] + str(temp["timestamp"]) + temp["dump"] + temp["segment"] + str(temp["image_urls"])[1:-1]
        #content = temp["content"] 
        text_list.append([index, temp])
        #text_list.append(content)
        #print(len(text_list))
    print(type(temp))
    return text_list

while True:
    try:
        a, b = next(generator)
    except StopIteration:
        break
    print("a: ", a, "b: ", b)


In [75]:
iterate_corpus()

<class 'dict'>


[[0,
  {'content': 'Postcards from a Global Trekker: Cape Town, South Africa\nI had a belated birthday celebration this year, given that my 40th birthday fell on Thanksgiving Day, 2010. While my family did a wonderful job incorporating a mini-celebration for me amongst the usual Turkey Day festivities, I never really did acknowledge the day with very much fanfare. Then six months into my 40th year I found myself presented with the rare opportunity of mixing business with pleasure and a second chance at marking a milestone year with a monumental event. At the coaxing of a colleague and friend, I said yes to taking a trip to a place that (20 years in the making) has been on my dream list of places to see in the world. This October I traveled to Cape Town, South Africa for the 2011 World Congress of the World Federation for Mental Health and below are just a few snapshots of the wonderful experience I had both professionally and personally.\nEleven days in total, half for the congress and